In [1]:
import math
import sympy
import numpy as np

In [2]:
def proba(params, gamma1, gamma2):
    n, l, eta_f, tau, td, _, k, Q = params
    return np.exp(-n*l*eta_f*tau*(l/(2**int(gamma1-1).bit_length()) +\
                                  (k*l*td)*(1/(2**int(gamma2-1).bit_length())+1/(Q-1))))

def repet(params, gamma1, gamma2):
    return 1/proba(params, gamma1, gamma2)
    
def entropy_c(param):
    n, l, _, tau= param[0:4]
    return l*int(math.comb(n, tau)*2**tau).bit_length()

def gamma1_(params):
    n, l, eta_f, tau = params
    return n*(l**2)*tau*eta_f

def gamma2_(params):
    n, l, eta_f, tau, td, tg, k, _ =params
    return n*(l**2)*k*td*tau*eta_f

def entropy_gd(param):
    n, l,_, __, td, tg, k = param[0:7]
    return l*k*(int(math.comb(n, td)*2**td).bit_length())+int(math.comb(n, tg)*2**tg).bit_length()

def entropy_g(param):
    n, l,_, __, td, tg, k = param[0:7]
    return int(math.comb(n, tg)*2**tg).bit_length()

def entropy_d(param):
    n, l,_, __, td, tg, k = param[0:7]
    return l*k*(int(math.comb(n, td)*2**td).bit_length())


def find_tau(n, l, top):
    t = 1
    while entropy_c(n, l, t) < top:
        t += 1

    return t

def find_td(n, l, k, top):
    t = 1
    while entropy_gd((n, l, None, None, t, t, k)) < top:
        t += 1

    return t

In [3]:
# params = (n, l, eta_f, tau, td, tg,  k, q)
params = {
   "2": (1024, 1, 1, 16, 7, 7, 1, 2021377),
   "5": (2048, 1, 1, 30, 13, 14, 1 , 33292289),
   "5++": (1024, 2, 1, 16, 3, 16, 3, 7340033)
}

In [4]:
for key in params.keys():
    print("{}: Entropy_c = {} \tEntropy_gd = {} \tEntropy_d = {} \tEntropy_g = {}".format(key, entropy_c(params[key]), entropy_gd(params[key]), entropy_d(params[key]), entropy_g(params[key])))

2: Entropy_c = 132 	Entropy_gd = 130 	Entropy_d = 65 	Entropy_g = 65
5: Entropy_c = 252 	Entropy_gd = 256 	Entropy_d = 124 	Entropy_g = 132
5++: Entropy_c = 264 	Entropy_gd = 318 	Entropy_d = 186 	Entropy_g = 132


In [5]:
for key in params.keys():
    print("{}: Repets = {}".format(key, repet(params[key][0:8], gamma1_(params[key][0:4]), gamma2_(params[key]))))

2: Repets = 6.901491782878755
5: Repets = 5.602482996658697
5++: Repets = 5.16991962954599


In [7]:
def generate_q1(maxi, n):
    init = maxi + 3
    alpha = int(2*n - 1).bit_length()
    ite = init - 1
    beg = 2**init - 2**ite + 1 
    while((not sympy.isprime(beg)) and alpha < ite):
        ite -= 1
        beg = 2**init - 2**ite + 1
        
    return (beg, ite) if sympy.isprime(beg) else None

In [9]:
def generate_q2(maxi, n):
    init = maxi + 3
    alpha = int(2*n - 1).bit_length()
    beta = 1
    beg = 2**init - beta * 2**alpha + 1 
    while(not sympy.isprime(beg) and beta < (2**(maxi+2-alpha))):
        beta += 2
        beg = (2**init) - beta * (2**alpha) + 1 
        
    return (beg, beta, alpha, init) if sympy.isprime(beg) else None